In [3]:
!pip install --upgrade Flask


In [4]:
import secrets

secrets.token_hex(32)

'0b89ceefdb281d45ab455ac354a9245a8a066ebb6b7210720328ed4583fb94dc'

In [2]:
## pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [9]:
import os

def list_files(startpath):
    for root, dirs, files in os.walk(startpath):
        level = root.replace(startpath, '').count(os.sep)
        indent = ' ' * 4 * (level)
        print('{}{}/'.format(indent, os.path.basename(root)))
        subindent = ' ' * 4 * (level + 1)
        for f in files:
            print('{}{}'.format(subindent, f))

# list_files(".")

'cat' is not recognized as an internal or external command,
operable program or batch file.


In [ ]:
docker run -d \
    --name my-python-app \
    -v /path/to/project/logs:/app/logs \
    -v /path/to/project/data:/app/data \
    my-python-app


In [ ]:
docker run -d --name job_scrap -v $(pwd)/logs:/app/logs -v $(pwd)/data:/app/data my-python-app


In [ ]:
docker run -d --name job_scrap \
    -v "C:/Users/MalikW/OneDrive/Project data/Job Scrap/logs:/app/logs" \
    -v "C:/Users/MalikW/OneDrive/Project data/Job Scrap/data:/app/data" \
    -v "C:/Users/MalikW/OneDrive/Project data/Job Scrap/credentials:/app/_secrets" \
    job_scrap



In [ ]:
docker run -d --name job_scrap `
    -v "C:/Users/MalikW/OneDrive/Project data/Job Scrap/logs:/app/logs" `
    -v "C:/Users/MalikW/OneDrive/Project data/Job Scrap/data:/app/data" `
    -v "C:/Users/MalikW/OneDrive/Project data/Job Scrap/credentials:/app/_secrets" `
    job_scrap


In [ ]:
docker run -p 8080:8080 -d --name job_scrap `
    -v "C:/Users/MalikW/OneDrive/Project data/Job Scrap/logs:/app/logs" `
    -v "C:/Users/MalikW/OneDrive/Project data/Job Scrap/data:/app/data" `
    -v "C:/Users/MalikW/OneDrive/Project data/Job Scrap/credentials:/app/_secrets" `
    job_scrap


In [ ]:
docker run -it --name job_scrap `
    -v "C:/Users/MalikW/OneDrive/Project data/Job Scrap/logs:/app/logs" `
    -v "C:/Users/MalikW/OneDrive/Project data/Job Scrap/data:/app/data" `
    -v "C:/Users/MalikW/OneDrive/Project data/Job Scrap/credentials:/app/_secrets" `
    job_scrap2 bash



In [1]:
!python --version

Python 3.13.1


In [5]:
pip freeze > requirements.txt


Note: you may need to restart the kernel to use updated packages.
